In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

 in practice, charector level token are rarely used. instead "chunks"/ bytepair etc

In [2]:
data = open('input.txt', 'r', encoding='utf-8').read()
len(data)

33396219

In [3]:
tokens = data.encode('utf-8')
len(tokens),len(set(tokens))

(33427740, 155)

In [4]:
text = open('input.txt','r',encoding='utf-8').read()
tokens = [ord(char) for char in text]
len(tokens)

33396219

In [5]:
def counts(id:list):
  count = {}
  for p in zip(id,id[1:]):
    count[p]  = count.get(p,0) + 1
  return count
  
def merge(id,top,tok):
  nid = []
  i = 0
  while i < len(id):
    if i < len(id) - 1 and id[i] == top[0] and id[i+1] == top[1]:
      nid.append(tok)
      i+=2
    else:
      nid.append(id[i])
      i+=1
  return nid
# count = counts(text)
# top = max(count,key=count.get)
# toke = merge(text,top,256)
# len(toke)

In [6]:
# it is tunable how many times to encode with bpe. but there are diminishing returns as vocab size will increase    this takes around 5 min for 20 merges on 3mil
vocab_size = 276  # 20 merges
num_merge = vocab_size - 256
ids = list(tokens) # copy

merges = {}    # t1,t2 -> t3
for i in range(num_merge):
  stats = counts(ids)
  pair = max(stats,key=stats.get)
  idx = 256 + i
  ids = merge(ids,pair,idx)
  merges[pair] = idx

KeyboardInterrupt: 

In [7]:
print(len(tokens)/len(ids)) , print(ids)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



(None, None)

In [8]:
print(merges)

{(101, 32): 256, (44, 32): 257, (46, 32): 258, (105, 110): 259, (116, 104): 260, (101, 114): 261, (32, 97): 262, (46, 10): 263, (111, 114): 264, (115, 32): 265, (111, 110): 266, (100, 32): 267, (97, 110): 268, (116, 32): 269, (101, 110): 270, (111, 32): 271, (32, 32): 272, (111, 102): 273, (263, 10): 274, (260, 256): 275}


In [9]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0,p1),idx in merges.items():    # needs and guaranteed after python 3.7 to be in order of insertion
  vocab[idx] = vocab[0] + vocab[1] # concat
def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8",errors='replace')  # for char like 128 replace with ?(unknown)
  return text


In [10]:
def encode(text):
  tokens = list(text.encode('utf-8'))
  while len(tokens)>=2:
    stats = counts(tokens)
    pair = min(stats,key=lambda p: merges.get(p,float('inf')))
    if pair not in merges:
      break
    idx = merges[pair]
    tokens = merge(tokens,pair,idx)
  return tokens

In [11]:
print(decode(encode("vocab = {idx: bytes([idx]) for idx in range(256)}for (p0,p1),idx in merges.items():    # needs and guaranteed after python 3.7 to be in order of insertionvocab[idx] = vocab[0] + vocab[1] # concatdef decode(ids):tokens = b"".join(vocab[idx] for idx in ids)text = token'# for char like 128 replace with ?(unknown)return text")))

vocab = {idx: bytes([idx]) f  idx   r ge(256)}f  (p0,p1),idx   m ges.items():  # needs n guar teed ft  py   3.7 t b    d     s ti vocab[idx] = vocab[0] + vocab[1] # c catdef decode(ids):tok  = b.jo (vocab[idx] f  idx   ids)tex = tok '# f  char lik 128 replac wi  ?(unknown)return text


This doesnt take into account abc followed by exclamations mean different i.e. abc. and abc! shouldnt be merged to x.,x!

In [12]:
import regex as re

pattern = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
print(re.findall(pattern,"this will've split this in words123"))       # in and ' words' will always be seperate instead of maybe 'n '

['this', ' will', "'ve", ' split', ' this', ' in', ' words', '123']


In [17]:
# special tokens
# 256 unicode, 50k merges, '<|endoftext|>'
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")  # gpt4

print(enc.decode(enc.encode("""print(re.findall(pattern,"this will've split this in words123"))       # in and ' words' will always be seperate instead of maybe 'n '""")))


print(re.findall(pattern,"this will've split this in words123"))       # in and ' words' will always be seperate instead of maybe 'n '


In [26]:
# import sentencepiece as spm      # train and inference of vocabs
# import os
# options = dict(
#   model_prefix="spm_model",
#   vocab_size=vocab_size,
# #...
# )
# spm.SentencePieceTrainer.Train(**options)
# sp = spm.SentencePieceProcessor()
# sp.load('spm.model')
# print(sp.EncodeAsPieces(text))